In [0]:
%tensorflow_version 1.x

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#importing all of the essential libraries
import cv2
from glob import glob
import matplotlib.pyplot as plt
from PIL import Image,ImageOps
import numpy as np
from keras.models import model_from_json
from keras.models import load_model
from keras.models import Model
from skimage.io import imread

Using TensorFlow backend.


***Extracting images from video with the gap of 1 second***

In [0]:
pathIn = '/content/drive/My Drive/extract.mp4'
pathOut = '/content/drive/My Drive/vishnu'

In [0]:
count = 0
vidcap = cv2.VideoCapture(pathIn)
success,image = vidcap.read()
success = True
k = []
while success:
  vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))    # added this line 
  success,image = vidcap.read()
  print ('Read a new frame: ', success)
  if success:
    k.append(image)    # save frame as JPEG file
  count = count + 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  False


In [0]:
from os import listdir
from os.path import isfile, join
import numpy
import cv2

mypath='/content/drive/My Drive/photos'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = numpy.empty(len(onlyfiles), dtype=object)
for n in range(0, len(onlyfiles)):
  images[n] = cv2.imread( join(mypath,onlyfiles[n]) )

In [3]:
k = [plt.imread(x) for x in glob('/content/drive/My Drive/photos/*.jpg')]
X = []
for i in range(len(k)):
  image = k[i]
  image = Image.fromarray(image)  
  img_dim = (160,160);
  image = ImageOps.fit(image,(img_dim[0],img_dim[1]),Image.ANTIALIAS,centering=(0.5, 0.5))
  image = np.array(image).reshape(img_dim[1],img_dim[0],3) 
  image  = np.array(image)
  X.append(image)

UnidentifiedImageError: ignored

In [0]:
k = [cv2.imread(x) for x in glob('/content/drive/My Drive/photos/*.jpg')]

In [0]:
X = []

In [7]:
for i in range(len(k)):
  image = k[i]
  #image = Image.fromarray(image)  
  img_dim = (160,160);
  image = ImageOps.fit(image,(img_dim[0],img_dim[1]),Image.ANTIALIAS,centering=(0.5, 0.5))
  image = np.array(image).reshape(img_dim[1],img_dim[0],3) 
  image  = np.array(image)
  X.append(image)

AttributeError: ignored

In [0]:
X[1].shape

(160, 160, 3)

In [0]:
facenet = load_model('/content/drive/My Drive/files/facenet_keras.h5')
facenet.load_weights('/content/drive/My Drive/files/facenet_keras_weights.h5')
facenet_ext = Model(inputs=facenet.input, outputs=facenet.get_layer('Mixed_7a').output)
facenet_ext.summary()

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
____________________________________________________________________________________________

In [0]:
for i in range(len(X)):
  X[i] = X[i].reshape((1,)+X[i].shape)
  print(X[i].shape)

(1, 160, 160, 3)
(1, 160, 160, 3)
(1, 160, 160, 3)
(1, 160, 160, 3)


In [0]:
embedding_images = []
for i in range(len(X)):
  embedding_images.append(facenet_ext.predict(X[i]))

In [0]:
# load json and create model
json_file = open('/content/drive/My Drive/files/facenet-1_train_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("/content/drive/My Drive/files/facenet-1_train_1.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
predicted = []
for i in range(len(X)):
  predicted.append(model.predict(embedding_images[i]))

In [0]:
k = [0,0,0,0,0]
def cal_OCEAN(index):
  for i in range(len(predicted)):
    k[index] = k[index] + predicted[i][0][index]
  return(k[index]/len(predicted))

In [0]:
o = cal_OCEAN(0)
c = cal_OCEAN(1)
e = cal_OCEAN(2)
a = cal_OCEAN(3)
n = cal_OCEAN(4)

In [0]:
print(o,c,e,a,n)

0.18883824907243252 0.34382031857967377 0.14005370810627937 0.17145424708724022 0.14329000934958458
